In [25]:
import pandas as pd
import numpy as np 
import string

#checks if a given word is of english language
def isEnglishLetter(s):
    try:
        s.encode('ascii')
        return 1
    except:
        return 0

#reads input csv file    
data = pd.read_csv("out.csv")

In [26]:
##preprocessing

#remove all rows with field of study and industry are undefined
#since those 2 information are the descriptive information for the study
data = data[(data['educations-field_of_study'] != "--undefined--") & (data['industry'] != "--undefined--")]

#gets the significant columns only
columns = ["location", "industry", "educations-degree", "educations-end_date", "educations-field_of_study", "educations-school-name", "positions-is-current", "num-connections"]
new_data = data[columns]

#changes all work status from undefined to False (assumption: if undefined then the person is not working)
new_data['positions-is-current'] = new_data['positions-is-current'].replace('--undefined--', 'FALSE')

#replace all undefined values to nan
new_data = new_data.replace("--undefined--",np.NaN)
new_data = new_data.reset_index()

#removes rows that are not english
is_englishLetter = []
for i in range(0, new_data.shape[0]):
    is_englishLetter.append(isEnglishLetter(new_data["industry"][i]))
new_data["is_english_letter"] = is_englishLetter
new_data = new_data[(new_data["is_english_letter"] == 1)]

#drop none english letters
del new_data["is_english_letter"]
new_data = new_data.reset_index()
del new_data["level_0"]
del new_data["index"]

#changes to UpperCase all values
for i in columns:
    new_data[i] = [str(j).upper() for j in new_data[i].tolist()]
new_data

/Users/zenitharnejo/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,location,industry,educations-degree,educations-end_date,educations-field_of_study,educations-school-name,positions-is-current,num-connections
0,"CHENNAI AREA, INDIA",INFORMATION TECHNOLOGY AND SERVICES,BE,31/12/2010,ELECTRONICS AND COMMUNICATION,THE MADRAS SEVA SADAN HIGHER SECONDARY SCHOOL,FALSE,1
1,"MUMBAI, MAHARASHTRA, INDIA",EDUCATION MANAGEMENT,MASTER OF BUSINESS ADMINISTRATION (M.B.A.),31/12/2010,MARKETING,MUMBAI UNIVERSITY,FALSE,13
2,"SARAWAK, MALAYSIA",EDUCATION MANAGEMENT,ISLAMIC EDUCATION,31/12/2006,ISLAMIC EDUCATION,UNIVERSITI MALAYA,TRUE,0
3,"PARIS AREA, FRANCE",PUBLIC RELATIONS AND COMMUNICATIONS,MASTER'S,31/12/2011,PUBLIC COMMUNICATION,INSTITUT D'ETUDES POLITIQUES DE LILLE,FALSE,112
4,"CHENNAI AREA, INDIA",AUTOMOTIVE,MCA,31/12/1998,COMPUTER APPLICATIONS,ST. JOSEPH COLLEGE CUDDALORE,FALSE,18
5,"CHENNAI AREA, INDIA",FINANCIAL SERVICES,PGDMM,31/12/2006,MARKETING,ANNAMALAI UNIVERSITY,TRUE,10
6,"CHENNAI AREA, INDIA",COMPUTER SOFTWARE,B.TECH,31/12/2009,INFORMATION TECHNOLOGY,ANNA UNIVERSITY,TRUE,56
7,"QUILON AREA, INDIA",ACCOUNTING,B.COM,31/12/2003,COMMERCE,KERALA UNIVERSITY,FALSE,2
8,"HASSAN AREA, INDIA","LEISURE, TRAVEL & TOURISM",BSC,31/12/2006,"PHYSICS,CHEMISTRY,MATHS",MANGALORE UNIVERSITY,FALSE,74
9,"BENGALURU AREA, INDIA",INFORMATION TECHNOLOGY AND SERVICES,MASTER OF TECHNOLOGY (M.TECH.),31/12/2009,COMPUTER SCIENCE,VISVESVARAYA TECHNOLOGICAL UNIVERSITY,FALSE,75


In [27]:
import enchant
import re

d = enchant.Dict("en_US")
industry = new_data["industry"]#.unique()

english_word = []
for i in range(0, len(industry)):
    value = industry[i]
    res = re.findall(r"[\w']+", value)

    j = 0
    flag = False
    for j in range(0, len(res)):
        if d.check(res[j]) == False:
            english_word.append(False)
            flag = True
            break
    if flag == False:
        english_word.append(True)

new_data["english_word"] = english_word
new_data = new_data[(new_data["english_word"] == True)]

#drop none english words
del new_data["english_word"]
new_data = new_data.reset_index()
del new_data["index"]
new_data

new_data.to_csv("out_file.csv", index=False, encoding='utf-8')

In [28]:
new_location = []

for i in range(0, new_data["location"].shape[0]):
    loc = new_data["location"][i].split(',')
    new_location.append(loc[len(loc)-1])

new_data["location"] = new_location
new_data.to_csv("out_file.csv", index=False, encoding='utf-8')
new_data

,location,industry,educations-degree,educations-end_date,educations-field_of_study,educations-school-name,positions-is-current,num-connections
0,INDIA,INFORMATION TECHNOLOGY AND SERVICES,BE,31/12/2010,ELECTRONICS AND COMMUNICATION,THE MADRAS SEVA SADAN HIGHER SECONDARY SCHOOL,FALSE,1
1,INDIA,EDUCATION MANAGEMENT,MASTER OF BUSINESS ADMINISTRATION (M.B.A.),31/12/2010,MARKETING,MUMBAI UNIVERSITY,FALSE,13
2,MALAYSIA,EDUCATION MANAGEMENT,ISLAMIC EDUCATION,31/12/2006,ISLAMIC EDUCATION,UNIVERSITI MALAYA,TRUE,0
3,FRANCE,PUBLIC RELATIONS AND COMMUNICATIONS,MASTER'S,31/12/2011,PUBLIC COMMUNICATION,INSTITUT D'ETUDES POLITIQUES DE LILLE,FALSE,112
4,INDIA,AUTOMOTIVE,MCA,31/12/1998,COMPUTER APPLICATIONS,ST. JOSEPH COLLEGE CUDDALORE,FALSE,18
5,INDIA,FINANCIAL SERVICES,PGDMM,31/12/2006,MARKETING,ANNAMALAI UNIVERSITY,TRUE,10
6,INDIA,COMPUTER SOFTWARE,B.TECH,31/12/2009,INFORMATION TECHNOLOGY,ANNA UNIVERSITY,TRUE,56
7,INDIA,ACCOUNTING,B.COM,31/12/2003,COMMERCE,KERALA UNIVERSITY,FALSE,2
8,INDIA,"LEISURE, TRAVEL & TOURISM",BSC,31/12/2006,"PHYSICS,CHEMISTRY,MATHS",MANGALORE UNIVERSITY,FALSE,74
9,INDIA,INFORMATION TECHNOLOGY AND SERVICES,MASTER OF TECHNOLOGY (M.TECH.),31/12/2009,COMPUTER SCIENCE,VISVESVARAYA TECHNOLOGICAL UNIVERSITY,FALSE,75


In [39]:
#philippine data cleaning
philippine_data = new_data[new_data["location"] == "PHILIPPINES"]
philippine_data = philippine_data.reset_index()
del philippine_data["index"]

philippine_data["educations-school-name"].replace("UP", "UNIVERSITY OF THE PHILIPPINES")
philippine_data["educations-school-name"].replace("XAVIER UNIVERSITY", "XAVIER UNIVERSITY")
philippine_data["educations-school-name"].replace("ST. PAUL UNIVERSITY", "ST. PAUL UNIVERSITY")
philippine_data["educations-school-name"].replace("POLYTECHNIC UNIVERSITY OF THE PHILIPPINES", "POLYTECHNIC UNIVERSITY OF THE PHILIPPINES")
philippine_data["educations-school-name"].replace("P. UNIVERSITY OF THE PHILIPPINES", "POLYTECHNIC UNIVERSITY OF THE PHILIPPINES")
philippine_data["educations-school-name"].replace("UNIVERSITY OF THE PHILIPPINES", "UNIVERSITY OF THE PHILIPPINES")
philippine_data["educations-school-name"].replace("T.U.P.", "TECHNOLOGICAL UNIVERSITY OF THE PHILIPPINES")
philippine_data["educations-school-name"].replace("SAINT PAUL UNIVERSITY", "SAINT PAUL UNIVERSITY")
philippine_data["educations-school-name"].replace("MSU-ILIGAN INSTITUTE OF TECHNOLOGY", "MSU-ILIGAN INSTITUTE OF TECHNOLOGY")
philippine_data["educations-school-name"].replace("MSU-IIT", "MSU-ILIGAN INSTITUTE OF TECHNOLOGY")

philippine_data = philippine_data[philippine_data["educations-school-name"] != "31/12/2011"]
philippine_data = philippine_data[philippine_data["educations-school-name"] != "31/12/1999"]
philippine_data = philippine_data[philippine_data["educations-school-name"] != "NAN"]
philippine_data = philippine_data[philippine_data["educations-school-name"] != "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"]

philippine_data["educations-school-name"].unique()

array(['NEW NORTHERN MINDANAO COLLEGES', 'UNIVERSITY OF SANTO TOMAS',
       'CENTRAL PHILIPPINE UNIVERSITY', 'UNIVERSITY OF THE PHILIPPINES',
       'POLYTECHNIC UNIVERSITY OF THE PHILIPPINES',
       'DE LA SALLE UNIVERSITY', 'UP', 'DAVAO DEL NORTE STATE COLLEGE',
       'INFORMATICS COMPUTER INSTITUTE', "ST. PAUL'S COLLEGE, QUEZON CITY",
       'UNIVERSITY OF THE EAST', 'UNIVERSITY OF SAN JOSE - RECOLETOS',
       'THE NATIONAL TEACHERS COLLEGE',
       'LYCEUM OF THE PHILIPPINES UNIVERSITY - BATANGAS',
       'ATENEO DE NAGA UNIVERSITY', 'AMA UNIVERSITY',
       'ATENEO DE DAVAO UNIVERSITY', 'SAINT LOUIS UNIVERSITY (PH)',
       'PAMANTASAN NG LUNGSOD NG PASAY',
       'DE LA SALLE - COLLEGE OF ST. BENILDE',
       'SAINT JOSEPH INSTITUTE OF TECHNOLOGY', 'SAN SEBASTIAN COLLEGE',
       'MINDANAO STATE UNIVERSITY', 'UNIVERSITY OF SAINT ANTHONY',
       'UNIVERSITY OF SAN JOSE RECOLLETOS', 'CEBU NORMAL UNIVERSITY',
       'ADAMSON UNIVERSITY', 'DAVAO MEDICAL SCHOOL FOUNDATION, INC.',

In [40]:
specializations = [{'id': 0, 'label': 'ACCOUNTING/FINANCE', 'word_contains': {'INSURANCE','FINANC', 'ACCOUNTING', 'BANKING', 'PHARMACEUTICALS'}},
                   {'id': 1, 'label': 'ADMIN/HUMAN RESOURCE', 'word_contains': {'ADMIN', 'HUMAN RESOURCE'}},
                   {'id': 2, 'label': 'ARTS/MEDIA', 'word_contains': {'MEDIA', 'ARTS', 'DESIGN'}},
                   {'id': 3, 'label': 'BUILDING/CONSTRUCTION', 'word_contains': {'BUILDING', 'CONSTRUCTION','CONCRETE','MINING'}},
                   {'id': 4, 'label': 'COMPUTER/INFORMATION TECHNOLOGY', 'word_contains': {'TELECOMMUNICATIONS','COMPUTER', 'INFORMATION','INTERNET'}},
                   {'id': 5, 'label': 'EDUCATION/TRAINING', 'word_contains': {'EDUCATION', 'TRAINING'}},
                   {'id': 6, 'label': 'ENGINEERING', 'word_contains': {'ENGINEERING','ARCHITECTURE'}},
                   {'id': 7, 'label': 'HEALTH', 'word_contains': {'MEDICINE', 'HEALTH','MEDICAL'}},
                   {'id': 8, 'label': 'HOTEL/RESTAURANT', 'word_contains': {'FOOD','HOSPITALITY','HOTEL', 'RESTAURANT'}},
                   {'id': 9, 'label': 'MANUFACTURING', 'word_contains': {'MANUFACTURING', 'PRODUCTION','PRINTING'}},
                   {'id': 10, 'label': 'SALES/MARKETING', 'word_contains': {'MARKETING', 'SALES'}},
                   {'id': 11, 'label': 'SCIENCES', 'word_contains': {'LIFE', 'SCIENCES', 'RESEARCH'}},
                   {'id': 12, 'label': 'JOURNALISM', 'word_contains': {'WRITING', 'JOURNALISM'}},
                   {'id': 13, 'label': 'ENTERTAINMENT', 'word_contains': {'ENTERTAINMENT'}},
                   {'id': 14, 'label': 'PUBLIC RELATIONS', 'word_contains': {'LEGAL','RELATIONS', 'LAW'}},
                   {'id': 15, 'label': 'BUSINESS', 'word_contains': {'TRADE', 'REAL'}},
                   {'id': 16, 'label': 'OTHERS', 'word_contains': {''}}]

spe = []
for i in philippine_data["industry"]:
    val_spe = [l["id"] for l in specializations if any(w in i for w in l["word_contains"])] 
    spe.append(val_spe[0])
    
philippine_data["specialization"] = spe

#changes all work status from undefined to False (assumption: if undefined then the person is not working)
philippine_data['positions-is-current'] = philippine_data['positions-is-current'].replace('FALSE', 0)
philippine_data['positions-is-current'] = philippine_data['positions-is-current'].replace('TRUE', 1)

del philippine_data["industry"]
del philippine_data["location"]

In [41]:
philippine_data.to_csv("philippine_file.csv", index=False, encoding='utf-8')
philippine_data

,educations-degree,educations-end_date,educations-field_of_study,educations-school-name,positions-is-current,num-connections,specialization
0,BACHELOR OF SCIENCE IN ELEMENTARY EDUCATION,31/12/2013,GENERAL,NEW NORTHERN MINDANAO COLLEGES,0,0,5
1,BACHELOR OF SCIENCE,NAN,HOTEL AND RESTAURANT MANAGEMENT,UNIVERSITY OF SANTO TOMAS,1,4,5
2,BSME,31/12/2012,MECHANICAL/INDUSTRY,CENTRAL PHILIPPINE UNIVERSITY,1,3,6
3,BSME,31/12/1987,ENGINEERING,UNIVERSITY OF THE PHILIPPINES,0,36,4
4,BACHELOR,31/12/1994,BANKING AND FINANCE,POLYTECHNIC UNIVERSITY OF THE PHILIPPINES,1,0,0
5,NAN,31/12/2002,NAN,UNIVERSITY OF THE PHILIPPINES,0,5,5
7,MASTERAL,31/12/2008,ENGLISH,DE LA SALLE UNIVERSITY,1,6,5
8,BS,31/12/2009,COMPUTER SCIENCE,UP,0,0,4
9,BACHELOR'S DEGREE,31/12/2006,BACHELOR OF SCIENCE IN INFORMATION TECHNOLOGY,DAVAO DEL NORTE STATE COLLEGE,0,8,4
10,NAN,31/12/2007,MULTIMEDIA AND THE INTERNET,INFORMATICS COMPUTER INSTITUTE,0,6,4
